In [1]:
# 1. Import Required Libraries
import pandas as pd
import numpy as np
from datetime import datetime
import random
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import ColumnDataSource
from bokeh.layouts import column
output_notebook()

In [5]:
# 2. Load Dataset (Simulated Streaming Source)
data = pd.read_csv("/content/dataset.csv")

In [7]:
# Create 'time' column from LastUpdatedDate + LastUpdatedTime
data['time'] = pd.to_datetime(data['LastUpdatedDate'] + ' ' + data['LastUpdatedTime'], dayfirst=True)



In [8]:
# Rename columns to expected names
data.rename(columns={
    'SystemCodeNumber': 'lot_id',
    'Occupancy': 'occupancy',
    'Capacity': 'capacity',
    'QueueLength': 'queue',
    'TrafficConditionNearby': 'traffic_level',
    'IsSpecialDay': 'special_day'
}, inplace=True)


In [15]:

# Convert key columns to numeric
for col in ['occupancy', 'capacity', 'queue', 'traffic_level', 'special_day']:
    data[col] = pd.to_numeric(data[col], errors='coerce').fillna(0)

In [9]:
#  Sort by time
data.sort_values('time', inplace=True)
data.reset_index(drop=True, inplace=True)


In [10]:
# 3. Add Initial Pricing Parameters
data['base_price'] = 10
lot_prices = {}
for lot_id in data['lot_id'].unique():
    lot_prices[lot_id] = 10  # initialize all lots at $10

In [11]:
# 4. Define Linear Pricing Model
def update_price_linear(row, alpha=0.5):
    occ_ratio = row['occupancy'] / row['capacity']
    return row['base_price'] + alpha * occ_ratio

In [22]:
# 5. Define Demand-Based Pricing Model
def calculate_demand_score(row):
    demand = (
        (row['occupancy'] / row['capacity']) +
        (row['queue'] / (row['capacity'] + 1)) +
        (row['traffic_level']) +
        (row['special_day'] * 0.5)
    )
    return demand / 3.5  # normalize demand score

def update_price_demand(row, base_price=10, lam=0.4):
    score = calculate_demand_score(row)
    return base_price * (1 + lam * score)


In [16]:
# 6. Simulate Streaming with Demand-Based Model
price_history = []
for index, row in data.iterrows():
    lot_id = row['lot_id']
    new_price = update_price_demand(row)
    lot_prices[lot_id] = new_price
    price_history.append({
        'time': row['time'],
        'lot_id': lot_id,
        'price': new_price
    })

price_df = pd.DataFrame(price_history)


In [23]:
# Compare Linear Pricing
linear_price_history = []
for index, row in data.iterrows():
    lot_id = row['lot_id']
    new_price = update_price_linear(row)
    linear_price_history.append({
        'time': row['time'],
        'lot_id': lot_id,
        'price': new_price
    })

linear_df = pd.DataFrame(linear_price_history)
linear_df.to_csv("/content/linear_price_output.csv", index=False)
print("✅ Linear pricing stream saved as linear_price_output.csv")


✅ Linear pricing stream saved as linear_price_output.csv


In [24]:

# Compare Linear Pricing
linear_price_history = []
for index, row in data.iterrows():
    lot_id = row['lot_id']
    new_price = update_price_linear(row)
    linear_price_history.append({
        'time': row['time'],
        'lot_id': lot_id,
        'price': new_price
    })

linear_df = pd.DataFrame(linear_price_history)
linear_df.to_csv("/content/linear_price_output.csv", index=False)
print("✅ Linear pricing stream saved as linear_price_output.csv")


✅ Linear pricing stream saved as linear_price_output.csv


In [25]:
# 7. Bokeh Visualization of Price Changes
plots = []
for lot_id in price_df['lot_id'].unique():
    subset = price_df[price_df['lot_id'] == lot_id]
    source = ColumnDataSource(subset)
    p = figure(title=f"Price Over Time for Lot {lot_id}", x_axis_type='datetime', width=700, height=300)
    p.line('time', 'price', source=source, line_width=2, color="navy", legend_label=f"Lot {lot_id}")
    p.xaxis.axis_label = 'Time'
    p.yaxis.axis_label = 'Price ($)'
    p.legend.label_text_font_size = '10pt'
    plots.append(p)

show(column(*plots))

In [26]:
# 8. Save Final Output (Optional)
price_df.to_csv("/content/final_submission.csv", index=False)
print("✅ Final pricing stream saved as final_submission.csv")


✅ Final pricing stream saved as final_submission.csv
